# Zad 10 - `Zadanie_DMD`

**Temat:** Dynamic Mode Decomposition (DMD)

 ## **Treść zadania**

Na podstawie danych o dynamicznych stanach układu (zgodnie z wariantem zadania) w macierzach $X$ oraz $X'$ obliczyć na podstawie metody DMD przybliżoną macierz przekształcenia $A$

**Wariant zadania: `14`**

### Kod Python

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd

rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = [10, 6]

# 1. WCZYTANIE DANYCH Z PLIKÓW CSV

# Wczytanie X (23 wiersze × 37 kolumn, pierwsza kolumna to indeks)
data_X = pd.read_csv('War14_X.csv', delimiter=';', decimal=',')
data_X = data_X.values

X = data_X.T  # po transpozycji: (36, 23)

# Wczytanie X' (23 wiersze × 36 kolumn)
data_Xprime = pd.read_csv('War14_Xprime.csv', delimiter=';', decimal=',')
data_Xprime = data_Xprime.values

Xprime = data_Xprime.T  # po transpozycji: (36, 23)

print("X shape:", X.shape)        # (36, 23)
print("Xprime shape:", Xprime.shape)  # (36, 23)

X shape: (36, 22)
Xprime shape: (36, 22)


In [2]:
# 2. IMPLEMENTACJA ALGORYTMU DMD
def DMD(X, Xprime, r):
    """
    Dynamic Mode Decomposition.
    X, Xprime : macierze danych (stany w kolumnach)
    r         : rząd redukcji (r ≤ min(n, m))
    Zwraca: Phi (mody), Lambda (wartości własne), b (współczynniki),
            Atilde (zredukowana macierz dynamiki), A_approx (przybliżona macierz A)
    """
    # Krok 1: SVD macierzy X
    U, Sigma, VT = np.linalg.svd(X, full_matrices=False)
    Ur = U[:, :r]
    Sigmar = np.diag(Sigma[:r])
    VTr = VT[:r, :]

    # Krok 2: Obliczenie zredukowanej macierzy Atilde
    Atilde = np.linalg.solve(Sigmar.T, (Ur.T @ Xprime @ VTr.T).T).T

    # Krok 3: Dekompozycja własna Atilde
    Lambda, W = np.linalg.eig(Atilde)  # Lambda to wektor, W to macierz wektorów własnych
    Lambda = np.diag(Lambda)  # przekształcenie na macierz diagonalną

    # Krok 4: Odtworzenie pełnych modów DMD (Φ)
    Phi = Xprime @ np.linalg.solve(Sigmar.T, VTr).T @ W

    # Współczynniki (amplitudy) modów
    alpha1 = Sigmar @ VTr[:, 0]
    b = np.linalg.solve(W @ Lambda, alpha1)

    # Obliczenie pełnej macierzy A (przybliżenie)
    A_approx = Xprime @ np.linalg.pinv(X)

    return Phi, Lambda, b, Atilde, A_approx

In [3]:
# 3. OBLICZENIA DLA r = 21
r = 21
Phi, Lambda, b, Atilde, A_approx = DMD(X, Xprime, r)

print("\n" + "="*60)
print("WYNIKI DMD")
print("="*60)
print(f"Zredukowana macierz Atilde ({Atilde.shape[0]}×{Atilde.shape[1]}):")
print(Atilde)


WYNIKI DMD
Zredukowana macierz Atilde (21×21):
[[ 1.21528665e+01  1.06701468e+02  1.32691802e+03 -2.42175166e+04
   1.23117679e+05  4.94039334e+06 -2.98211730e+07  4.88325503e+06
   3.97764233e+09  3.30964248e+10  1.51873336e+11  5.27971358e+10
   3.46897821e+12 -2.59590200e+13  1.33550887e+14 -4.14199255e+15
   5.27660916e+17  8.74433301e+17 -3.06537375e+18  1.74188153e+19
  -3.91291626e+19]
 [ 2.05831233e-07 -8.82048646e+00 -1.09550464e+02  1.99939536e+03
  -1.01645812e+04 -4.07878298e+05  2.46203256e+06 -4.03160965e+05
  -3.28393686e+08 -2.73243697e+09 -1.25386449e+10 -4.35892537e+09
  -2.86398436e+11  2.14317366e+12 -1.10259456e+13  3.41962422e+14
  -4.35636236e+16 -7.21931113e+16  2.53076899e+17 -1.43809536e+18
   3.23049910e+18]
 [ 4.45458073e-08  9.00734978e-01 -1.25725805e+00  2.24199218e+01
  -1.13986160e+02 -4.57397222e+03  2.76093840e+04 -4.52107175e+03
  -3.68262691e+06 -3.06417155e+07 -1.40609132e+08 -4.88812561e+07
  -3.21168961e+09  2.40336808e+10 -1.23645630e+11  3.834

In [9]:
print("\nWartości własne A (diagonalne):")
print(np.diag(Lambda))


Wartości własne A (diagonalne):
[ 5.06541138e+00+26.52057661j  5.06541138e+00-26.52057661j
  1.58410134e+01 +0.j          1.21528646e+01 +0.j
  5.91826040e+00+11.80929035j  5.91826040e+00-11.80929035j
  8.58077554e+00 +7.65805853j  8.58077554e+00 -7.65805853j
 -1.47382145e+01 +0.j         -1.23537164e+01 +0.j
 -9.58764550e+00 +5.4480609j  -9.58764550e+00 -5.4480609j
 -4.34456238e-01+11.06396454j -4.34456238e-01-11.06396454j
 -5.61211456e+00 +9.44976915j -5.61211456e+00 -9.44976915j
  6.47804140e+00 +0.j         -3.11598531e+00 +0.j
 -8.03546207e-02 +0.j         -1.01225200e-09 +0.j
 -9.69960800e-08 +0.j        ]


In [8]:
print(f"\nPrzybliżona macierz A ({A_approx.shape[0]}×{A_approx.shape[1]}):")
print(A_approx)



Przybliżona macierz A (36×36):
[[ 5.41686283e-28  1.08608626e-34  3.07788698e-36 ...  4.47510028e-31
   1.50945544e-32 -4.59312054e-34]
 [-1.06158220e-27 -2.31133408e-34 -6.50010713e-36 ...  1.53556261e-30
  -4.45419222e-32  1.00155278e-33]
 [-6.08880805e-28 -1.01927510e-34 -2.94300308e-36 ... -1.98372627e-30
   3.98545437e-32 -2.44505072e-33]
 ...
 [ 2.02715794e-06  1.19346908e-12  1.64309790e-15 ...  4.61002065e-10
   1.00000000e+00  5.87771452e-13]
 [ 2.97826094e-05  1.79007608e-12  7.13844975e-14 ...  8.68344364e-10
   4.98822871e-10  1.00000000e+00]
 [ 1.41267015e+07  3.36177728e+00  9.38485915e-02 ...  2.69954983e+02
  -9.88060785e+01  1.41096173e+00]]
